In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
import joblib

In [2]:
DATASET_DIR = "./dataset.csv"
LABEL_CHROMA_FEATURE = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
LABEL_CHORD = 'chord'

In [3]:
df = pd.read_csv(DATASET_DIR)
df

,C,Db,D,Eb,E,F,Gb,G,Ab,A,Bb,B,chord
0,0.183537,0.058886,0.074500,0.148481,0.231460,1.000000,0.195419,0.086401,0.050765,0.033882,0.016108,0.056176,FMin7
1,0.183537,0.058886,0.074500,0.148481,0.231460,1.000000,0.195419,0.111199,0.050765,0.033882,0.016108,0.056176,FMin7
2,0.194397,0.053142,0.069541,0.125648,0.218405,1.000000,0.172237,0.111199,0.050765,0.031130,0.016108,0.056176,FMin7
3,0.202456,0.053142,0.069541,0.125648,0.218405,1.000000,0.172237,0.111199,0.050765,0.031130,0.016108,0.056176,FMin7
4,0.213011,0.051345,0.068525,0.121922,0.210554,1.000000,0.140933,0.111199,0.050765,0.031130,0.015734,0.051489,FMin7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26641,1.000000,0.084976,0.331198,0.042095,0.051681,0.014662,0.058985,0.016012,0.013100,0.045549,0.013784,0.140476,D7
26642,1.000000,0.080510,0.322723,0.042095,0.051681,0.021371,0.051865,0.019967,0.014216,0.043450,0.023052,0.140476,D7
26643,1.000000,0.080510,0.314526,0.042173,0.051681,0.021371,0.040013,0.020209,0.016708,0.043450,0.037827,0.140476,D7
26644,1.000000,0.080510,0.255922,0.042173,0.051681,0.021371,0.037740,0.022714,0.016903,0.035687,0.037827,0.140476,D7


In [4]:
df_copy = df.copy()
y = df_copy["chord"]
X = df_copy.drop(columns="chord")
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=42, shuffle=True, test_size=0.2, stratify=y_encoded)

In [5]:
print(f"X_train: {len(X_train)}")
print(f"X_test: {len(X_test)}")

X_train: 21316
X_test: 5330


In [6]:
joblib.dump(encoder, "./encoder.xz")

['./encoder.xz']

# Models

## SVM

In [ ]:
svc = SVC()
params_svm = [{
    "C": [1, 10, 1000],
    # "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "kernel": ['poly', 'rbf', 'sigmoid'],
    # "gamma": ['scale', 'auto', 1, 0.1, 0.01, 0.001, 0.0001]
    "gamma": [1, 0.1, 0.01, 0.001, 0.0001]
}]
grid_search_svm = GridSearchCV(svc, params_svm, cv=5)

In [ ]:
grid_search_svm.fit(X_train, y_train)

In [ ]:
print(f"Best params: {grid_search_svm.best_params_}")
print(f"Best CV score: {grid_search_svm.best_score_ * 100}%")
model_svm = grid_search_svm.best_estimator_

In [ ]:
y_pred = model_svm.predict(X_test)
score = accuracy_score(y_pred, y_test)
print(f"Model accuracy: {score * 100}%")

In [ ]:
joblib.dump(model_svm, "./model_svm.xz")
joblib.dump(grid_search_svm, "./grid_search_svm.xz")

In [ ]:
knn = KNeighborsClassifier()
params_knn = [{
    "n_neighbors": [2, 3, 4, 5, 6],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [10, 20, 30, 40, 50]
}]
grid_search_knn = GridSearchCV(knn, params_knn)

## KNN

In [ ]:
grid_search_knn.fit(X_train, y_train)

In [ ]:
print(f"Best params: {grid_search_knn.best_params_}")
print(f"Best CV score: {grid_search_knn.best_score_ * 100}%")
model_knn = grid_search_knn.best_estimator_
# score = accuracy_score(y_pred, y_test)
# print(f"Model accuracy: {score * 100}%")

## Random Forest

In [7]:
rf = RandomForestClassifier()
params_rf = [{
    "n_estimators": [100, 150, 200, 250, 300, 350],
    "max_depth": [None, 1, 2, 3, 4, 5],
    "warm_start": [True, False]
}]
grid_search_rf = GridSearchCV(rf, params_rf)

In [8]:
grid_search_rf.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [None, 1, 2, 3, 4, 5],
                          'n_estimators': [100, 150, 200, 250, 300, 350],
                          'warm_start': [True, False]}])

In [9]:
print(f"Best params: {grid_search_rf.best_params_}")
print(f"Best CV score: {grid_search_rf.best_score_ * 100}%")
model_rf = grid_search_rf.best_estimator_

Best params: {'max_depth': None, 'n_estimators': 350, 'warm_start': False}
Best CV score: 93.97634275292572%


In [10]:
y_pred = model_rf.predict(X_test)
score = accuracy_score(y_pred, y_test)
print(f"Model accuracy: {score * 100}%")

Model accuracy: 94.70919324577861%


In [11]:
joblib.dump(model_rf, "./model_rf.xz")
joblib.dump(grid_search_rf, "./grid_search_rf.xz")

['./grid_search_rf.xz']